<a href="https://colab.research.google.com/github/Muthon1/DataScience/blob/main/Face_detection_app_Checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip install streamlit opencv-python-headless

In [4]:
!pip install streamlit opencv-python-headless pyngrok

In [5]:
# Install ngrok using apt package manager
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
!sudo apt update
!sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,646 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [

In [15]:
# Step 2: Write the Streamlit app code to a Python file
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image

# Load the pre-trained Viola-Jones face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Instructions
st.write("## Face Detection App")
st.write("Upload an image and adjust the parameters to detect faces using the Viola-Jones algorithm.")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Convert the file to an OpenCV image
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the uploaded image
    st.image(image_rgb, caption='Uploaded Image', use_column_width=True)

    # Color picker for rectangle
    color = st.color_picker('Pick a color for the rectangle', '#00FF00')
    color_rgb = tuple(int(color[i:i+2], 16) for i in (1, 3, 5))

    # Slider for minNeighbors
    min_neighbors = st.slider('minNeighbors', 1, 10, 5)

    # Slider for scaleFactor
    scale_factor = st.slider('scaleFactor', 1.01, 2.0, 1.1)

    # Detect faces
    faces = face_cascade.detectMultiScale(image, scaleFactor=scale_factor, minNeighbors=min_neighbors)

    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image_rgb, (x, y), (x+w, y+h), color_rgb, 2)

    # Display the image with detected faces
    st.image(image_rgb, caption='Detected Faces', use_column_width=True)

    # Save the image
    if st.button('Save Image'):
        cv2.imwrite('detected_faces.jpg', cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
        st.write("Image saved as detected_faces.jpg")

# Step 3: Run the Streamlit app with ngrok
from pyngrok import ngrok


# Create a new tunnel
public_url = ngrok.connect(port='8501')
print('Public URL:', public_url)


# Keep the cell running to maintain the ngrok tunnel
import time
while True:
    time.sleep(1)

Overwriting app.py


In [16]:
from pyngrok import ngrok

# Install Ngrok authtoken
!ngrok authtoken 2s1Pw5f1wJsL15HDSYJ8nZsrC6c_2A3sQwbgC6HVFUZFnFgNq

# Run Streamlit app
!streamlit run app.py &>/dev/null&

# Instead of creating a new tunnel, retrieve the public URL from the existing tunnel
# created in app.py using ngrok.get_tunnels()
tunnels = ngrok.get_tunnels()
public_url = tunnels[0].public_url  # Assuming the first tunnel is the one you want

# Print the public URL to access the app
print(public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ERROR:pyngrok.process.ngrok:t=2025-01-23T10:36:21+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-01-23T10:36:21+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [17]:
# Terminate open tunnels if any
ngrok.kill()

In [18]:
!ngrok http --url=emu-magnetic-extremely.ngrok-free.app 80

http - start an HTTP tunnel

USAGE:
  ngrok http [address:port | port] [flags]

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your account at https://dashboard.ngrok.com/b

In [19]:
# Run Streamlit app
!streamlit run app.py &>/dev/null&